In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow.keras.utils    import to_categorical
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers


import os, shutil



def image_checker(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass

original_dir = '.../input/dogs-vs-cats-redux-kernels-edition'
original_zip_dataset_dir = '../input/dogs-vs-cats-redux-kernels-edition/train.zip'
original_zip_test_dir = '../input/dogs-vs-cats-redux-kernels-edition/test.zip'
unzipped_base_dir = '../working/cats_and_dogs_full_unzipped'
unzipped_train = os.path.join(unzipped_base_dir, 'train')
unzipped_test = os.path.join(unzipped_base_dir, 'test')
original_cats_dataset_dir = unzipped_train
original_dogs_dataset_dir = unzipped_train
original_test_dir = os.path.join(unzipped_test, 'test')
base_dir = '../working/cats_and_dogs_full'
if os.path.exists(base_dir) and os.path.isdir(base_dir):
	shutil.rmtree(base_dir)



import zipfile
with zipfile.ZipFile(original_zip_dataset_dir,"r") as z:
    z.extractall(unzipped_base_dir)
with zipfile.ZipFile(original_zip_test_dir,"r") as z:
    z.extractall(unzipped_test)

    
list = os.listdir(unzipped_train)
number_files = len(list)
print("Number of files in the trainining set:",number_files)
list = os.listdir(original_test_dir) 
number_files = len(list)
print("Number of files in the test set:",number_files)


os.mkdir(base_dir)
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)


fnames = ['cat.{}.jpg'.format(i) for i in range(8000)]
for fname in fnames:
	src = os.path.join(original_cats_dataset_dir, fname)
	if os.path.exists(src):
		dst = os.path.join(train_cats_dir, fname)
		shutil.copyfile(src,dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(8000, 12500)]
for fname in fnames:
	src = os.path.join(original_cats_dataset_dir, fname)
	if os.path.exists(src):
		dst = os.path.join(validation_cats_dir, fname)
		shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(8000)]
for fname in fnames:
	src = os.path.join(original_dogs_dataset_dir, fname)
	if os.path.exists(src):
		dst = os.path.join(train_dogs_dir, fname)
		shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(8000, 12500)]
for fname in fnames:
	src = os.path.join(original_dogs_dataset_dir, fname)
	if os.path.exists(src):
		dst = os.path.join(validation_dogs_dir, fname)
		shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(8000, 12500)]
for fname in fnames:
	src = os.path.join(original_dogs_dataset_dir, fname)
	if os.path.exists(src):
		dst = os.path.join(validation_dogs_dir, fname)
		shutil.copyfile(src, dst)
fnames = ['{}.jpg'.format(i+1) for i in range(12500)]
for fname in fnames:
	src = os.path.join(original_test_dir, fname)
	if os.path.exists(src):
		dst = os.path.join(test_dir, fname)
		shutil.copyfile(src, dst)
        


train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,
                                   shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(300, 300),batch_size=40,class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(300, 300),batch_size=40,class_mode='binary')
test_generator = test_datagen.flow_from_directory(base_dir,target_size=(300, 300),batch_size=40,classes=['test'])


conv_base = InceptionV3(weights='imagenet',include_top=False, input_shape=(300,300,3), pooling="avg")
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
print(model.summary())
conv_base.trainable = False
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(learning_rate=1e-4),metrics=['acc'])

history = model.fit(image_checker(train_generator),steps_per_epoch=400,epochs=30,validation_data=validation_generator,validation_steps=225)

model.save('cats_and_dogs_large_12.h5')


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()


predictions = model.predict(test_generator, steps=313)
predictions = predictions.reshape(1,-1)[0]

pid = []
labels = []
for i in range(1,len(predictions)+1):
	pid.append(i)
	if (predictions[i-1] > 0.5):
		labels.append(1)
	else:
		labels.append(0)

output  = pd.DataFrame({'id': pid, 'label': labels})
output.to_csv('my_submission_code12.csv',index=False)

if os.path.exists(base_dir) and os.path.isdir(base_dir):
	shutil.rmtree(base_dir)
